In [1]:
from llama_index.core import (
    VectorStoreIndex,
    load_index_from_storage,
    StorageContext
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.settings import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
from llama_parse import LlamaParse

import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import nest_asyncio
nest_asyncio.apply()

## Instantiate objects

In [2]:
Settings.llm = Ollama(model="tinyllama")
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [3]:
reranker = FlagEmbeddingReranker(
    top_n = 2,
    model = "BAAI/bge-reranker-large"
)

In [4]:
splitter = SentenceSplitter(chunk_size = 1024, chunk_overlap = 200)

### Parse documents

In [3]:
parser = LlamaParse(
    api_key = os.environ.get("LLAMACLOUD"),
    result_type = "markdown",
    num_workers = 4,
    verbose = True,
    language = "en"
)

In [5]:
documents = parser.load_data(
    "../../../Lim Hsien Yong (Titus) Resume.pdf"
)

Started parsing the file under job_id 81ee58d9-ada6-49af-9121-3406953d70ee


In [13]:
documents[0].text[:1000]

'## LIM HSIEN YONG (“TITUS”)\n\ntituslhy@gmail.com • +65 9092 6178 • linkedin.com/tituslim • github.com/tituslhy • tituslim.onrender.com\n\n### EDUCATION\n\n|SINGAPORE MANAGEMENT UNIVERSITY|Aug 2021 - Dec 2022|\n|---|---|\n|Master of IT in Business (Artificial Intelligence Track)|Dean’s List, GPA: 3.98 / 4.0|\n|- Awarded the SMU AI Talent Development Grant and SMU MITB Scholarship| |\n\n|NATIONAL UNIVERSITY OF SINGAPORE|Aug 2011 - Jul 2015|\n|---|---|\n|Bachelor of Engineering in Chemical Engineering (2nd Upper Honors)| |\n|- Awarded the NUS Undergraduate Scholarship, Barco-Santander Scholarship and IE Singapore Young Talent Program Market Immersion Award| |\n\n### EXPERIENCE\n\n|Illumina – Singapore|Jan 2023 – Present|\n|---|---|\n|Senior Data Scientist| |\n|• Retrieval Augmented Generation (RAG):| |\n|o Developed an AI-powered assistant using LLaMA2, leveraging advanced retrieval techniques for efficient equipment troubleshooting and document comprehension. Line engineers now chat di

In [71]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core.node_parser import SentenceSplitter

storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore(),
    vector_store=SimpleVectorStore(),
    index_store=SimpleIndexStore(),
)

In [72]:
index = VectorStoreIndex.from_documents(documents, 
                                        storage_context = storage_context,
                                        transformations=[splitter])

Save data

In [73]:
index.storage_context.persist(persist_dir = "../data")

Load data from source

In [6]:
storage_context = StorageContext.from_defaults(persist_dir = "../database")

In [7]:
index = load_index_from_storage(storage_context)
query_engine = index.as_query_engine(
    similarity_top_k = 6,
    node_postprocessors=[reranker]
)

Get article

In [40]:
from llama_index.readers.web import SimpleWebPageReader
html_documents = SimpleWebPageReader(html_to_text=True).load_data(
    [
    # "https://hbr.org/2012/10/data-scientist-the-sexiest-job-of-the-21st-century",
    "https://www.google.com/search?q=data+scientist++jobs&newwindow=1&sca_esv=8aeb3926740031f7&rlz=1C5CHFA_enSG1033SG1035&sxsrf=ACQVn08vr42DJkrrK5vf2GNjiAJEoEuFGA:1711256630217&ei=NrT_ZavgDK6YjuMPqaiYiAE&uact=5&oq=google+singapore+data+scientist+jobs&gs_lp=Egxnd3Mtd2l6LXNlcnAiJGdvb2dsZSBzaW5nYXBvcmUgZGF0YSBzY2llbnRpc3Qgam9iczIFECEYkgMyBRAhGJIDMgUQIRiSAzIFECEYkgMyBRAhGJIDMgUQIRiSAzIFECEYnwVIqR1Q8wJY2xxwAXgBkAEAmAF0oAGDEaoBBDM1LjG4AQPIAQD4AQGYAiWgArYRwgIKEAAYRxjWBBiwA8ICDRAAGIAEGIoFGEMYsAPCAg0QLhiABBiKBRhDGLADwgIEECMYJ8ICChAjGIAEGIoFGCfCAgoQABiABBiKBRhDwgIWEC4YgAQYigUYQxixAxiDARjHARjRA8ICEBAAGIAEGIoFGEMYsQMYgwHCAg0QABiABBiKBRhDGLEDwgILEAAYgAQYigUYkQLCAhYQLhiABBgUGIcCGLEDGIMBGMcBGK8BwgINEAAYgAQYFBiHAhixA8ICCxAAGIAEGLEDGIMBwgIIEAAYgAQYsQPCAgUQABiABMICBBAAGAPCAhEQLhiABBixAxiDARjHARivAcICChAAGIAEGBQYhwLCAgYQABgWGB7CAgsQABiABBiKBRiGA8ICBRAhGKABmAMAiAYBkAYKkgcEMzYuMaAH0YgC&sclient=gws-wiz-serp&ibp=htl;jobs&sa=X&ved=2ahUKEwi9y-7mj4yFAxUh9DgGHeVcB7QQkd0GegQIIhAB#fpstate=tldetail&htivrt=jobs&htidocid=hR_pshAhVwJRf-kgAAAAAA%3D%3D"
    ]
)

In [41]:
html_query_engine = VectorStoreIndex.from_documents(
    html_documents, 
    transformations = [splitter])\
    .as_query_engine(similarity_top_k = 6, 
                     node_postprocessors=[reranker])

In [42]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="Titus resume",
            description=(
                "Provides information about Titus' employment history, skills,"
                ", educational qualifications and accomplishments."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=html_query_engine,
        metadata=ToolMetadata(
            name="Job description",
            description=(
                "Provides information about the data scientist job position at MINDEF "
            ),
        ),
    ),
]

## ReActRAG

In [43]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    query_engine_tools,
    verbose = True
)

In [12]:
response = agent.chat("What are Titus' core skillsets?")
print(str(response))

Thought: (Implicit) I can answer without any more tools!
Answer: Titus' core skillsets are listed below:

1. Data Science: Titus has expertise in data science, specifically in using data to solve problems and find insights. He is proficient in Python, R, Tableau, and SQL.

2. Business Analysis: Titus possesses a thorough understanding of business processes and their underlying requirements for success. He is skilled in analyzing customer needs, market trends, competitor analysis, and profitability assessment.

3. Technical Writing: Titus has excellent technical writing skills that enable him to write clear, concise documentation while adhering to industry standards. He can design and deliver training materials for technical topics, including software applications and hardware systems.

4. Project Management: As a project manager, Titus is well-versed in managing projects across various stages from conception to delivery, ensuring that objectives are met on time and within budget.
Titus

In [13]:
query = "What domains of AI does Titus specialize in?"
response = agent.chat(query)
print(response)

Thought: (Implicit) I can answer without any more tools!
Answer: Titus specializes in the following domains of Artificial Intelligence (AI):

1. Natural Language Processing: Titus has extensive experience with natural language processing, specifically in the areas of text analysis, sentiment analysis, and machine translation. He is proficient in Python, Natural Language Toolkit (NLTK), SpaCy, and BERT.

2. Machine Learning: Titus has expertise in machine learning algorithms and their applications to solve real-world problems. He specializes in building predictive models using various techniques such as random forests, support vector machines, and neural networks.

3. Deep Learning: Titus is proficient in deep learning frameworks such as PyTorch, TensorFlow, and Keras. He has expertise in building convolutional neural networks (CNNs) for image recognition and language modeling.

4. Reinforcement Learning: Titus also specializes in reinforcement learning for automated decision making in 

In [44]:
query = "What are the core skillsets required by MINDEF?"
response = agent.chat(query)
print(response)

Thought: (Implicit) I can answer without any more tools!
Answer: Here are some of the core skillssets required by MINDEF:

1. Data Science: MINDEF requires a strong background in data science, including knowledge of statistics, algorithms, and machine learning. This includes expertise in Python, R, or similar languages, as well as familiarity with data analysis techniques such as regression models, clustering, and dimensional reduction.

2. Artificial Intelligence: MINDEF requires a deep understanding of artificial intelligence (AI) principles and practices, including neural networks, machine learning algorithms, and deep learning. This includes proficiency in Python or other AI-focused programming languages, as well as familiarity with AI models such as decision trees, SVMs, or convolutional neural networks.

3. Data Visualization: MINDEF requires expertise in data visualization techniques, including plotting, mapping, and exploratory analysis. This includes proficiency in Python or s

In [46]:
query = "Is Titus a good fit for the job at MINDEF? Why or why not?"
response = agent.chat(query)
print(response)

Thought: (Implicit) I can answer without any more tools!
Answer: Tituz is indeed a suitable candidate for the job at MIND EF, based on their qualifications and experience. As a data science expert in the field of machine learning, Tituz has the necessary technical skills and analytical aptitude to handle complex data analyses and develop predictive models for MIND EF's clients.

Moreover, Titus's long-term track record of successful collaborations with major corporate clients makes them a good fit for this position at MIND EF. They have worked on various projects for MIND EF, which indicates a proven ability to deliver high-quality data analysis services. 

The organization prioritizes excellence and innovation in their research projects, aligning well with Titus's experience and expertise in data science, machine learning, and AI. Furthermore, MIND EF values long-term relationships, and Titus has a strong reputation as a reliable partner who can build trust with the organization. 

In

In [47]:
query = "What are Titus' best skillsets that overlap with the job description?"
response = agent.chat(query)
print(response)

Thought: (Implicit) I can answer without any more tools!
Answer: Tituz's best skillsets that overlap with the job description are:

1. Data Science: MIND EF requires a strong background in data science, including knowledge of statistics, algorithms, and machine learning principles. Titus has proficiency in Python or similar software packages for visualization, modeling, and analyses. They can apply these skills to work on complex data analyse projects at MIND EF.

2. Machine Learning: MIND EF prioritizes machine learning in their research projects. Titus's expertise in machine learning algorithms such as decision trees, regression analysis, and clustering can be leveraged to develop predictive models for clients.

3. Artificial Intelligence: AI is a core focus area at MIND EF. Titus has proficiency in AI principles, including the use of tools such as neural networks, machine learning algorithms, and deep learning. They can apply these skills to work on innovative AI solutions for clien